<a href="https://colab.research.google.com/github/KTH-EXPECA/examples/blob/main/sdr/setup_sdr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Authentication and Dependencies

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [ ]:
import os, re
from getpass import getpass

with open('/content/demo_project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········


Install required packages and dependencies. Ignore the warnings.

In [ ]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 952.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.3/324.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.1/256.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/253.7 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.5/515.5 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

Import packages

In [ ]:
import json, time
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, show_reservation_byname, restart_sdr, make_sdr_ni, make_sdr_mango, sdr_tools, get_available_publicips

# Reserve resources

In the next cell, we reserve 1 SDR and 1 worker.

In [ ]:
sdr_name = "sdr-07"
rj45_seg_id = "113"
sfp_seg_id = "114"

# reserve RJ45 port
rj45_lease = show_reservation_byname(sdr_name + "-rj45-lease")
if not rj45_lease:
    rj45_lease = reserve(
        { "type":"network", "name": sdr_name+"-rj45", "net_name": sdr_name+"-rj45", "segment_id": rj45_seg_id, "duration": { "days":7, "hours":0 } }
    )

# reserve SFP port
sfp_lease = show_reservation_byname(sdr_name + "-sfp-lease")
if not sfp_lease:
    sfp_lease = reserve(
        { "type":"network", "name": sdr_name+"-sfp", "net_name": sdr_name+"-sfp", "segment_id": sfp_seg_id, "duration": { "days":7, "hours":0 } }
    )

worker_name = 'worker-10'
# reserve worker
worker_lease = show_reservation_byname(worker_name+"-lease")
if not worker_lease:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
    )
worker_reservation_id = worker_lease["reservations"][0]["id"]


leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

2024-07-10 16:55:31.462 | INFO     | chi.expeca:reserve:167 - reserving sdr-07-rj45
2024-07-10 16:55:33.977 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for sdr-07-rj45-lease with id fa704f85-1766-4540-9082-f44712992f4f to become "ACTIVE"
2024-07-10 16:55:39.161 | INFO     | chi.expeca:wait_until_lease_status:69 - lease sdr-07-rj45-lease with id fa704f85-1766-4540-9082-f44712992f4f is PENDING.
2024-07-10 16:55:44.342 | INFO     | chi.expeca:wait_until_lease_status:69 - lease sdr-07-rj45-lease with id fa704f85-1766-4540-9082-f44712992f4f is PENDING.
2024-07-10 16:55:49.515 | INFO     | chi.expeca:wait_until_lease_status:69 - lease sdr-07-rj45-lease with id fa704f85-1766-4540-9082-f44712992f4f is PENDING.
2024-07-10 16:55:54.693 | INFO     | chi.expeca:wait_until_lease_status:69 - lease sdr-07-rj45-lease with id fa704f85-1766-4540-9082-f44712992f4f is PENDING.
2024-07-10 16:55:59.863 | INFO     | chi.expeca:wait_until_lease_status:69 - lease sdr-07-rj45-lease 

[
    {
        "name": "sdr-07-rj45-lease",
        "id": "473486fd-5802-4698-8e2a-cff3b33da095",
        "reservation_id": "d20789fb-8e2c-4365-8ab8-a8b9ea640280",
        "status": "ACTIVE",
        "end_date": "2024-07-17T16:55:00.000000"
    },
    {
        "name": "worker-10-lease",
        "id": "4f0d9954-ab0a-4c69-a897-fe88c27c6812",
        "reservation_id": "af122e0b-f89f-4efb-be0c-b1caa3e20d66",
        "status": "ACTIVE",
        "end_date": "2024-07-17T16:41:00.000000"
    },
    {
        "name": "sdr-07-sfp-lease",
        "id": "5b8cbe75-6ea9-46c6-9182-71f3cbace7c1",
        "reservation_id": "3e7b07f2-707e-4504-bcad-3f07c052be25",
        "status": "ACTIVE",
        "end_date": "2024-07-17T16:56:00.000000"
    }
]


# Configure the SDRs

## Change the sdrs' firmwares to Mango or NI

In [ ]:
# change sdr design to ni using the reserved worker
sdrnet = chi.network.get_network(sdr_name+"-rj45-net")
make_sdr_ni(sdr_name,sdrnet['id'],worker_reservation_id,"ens1")

# wait 10 seconds
#time.sleep(10)

# change sdr design to mango using the reserved worker
#sdrnet = chi.network.get_network(sdr_name+"-rj45-net")
#make_sdr_mango(sdr_name,sdrnet['id'],worker_reservation_id,"ens1")

2023-09-01 08:04:04.639 | SUCCESS  | chi.expeca:make_sdr_mango:299 - created make-sdr-mango container.
2023-09-01 08:04:04.641 | INFO     | chi.expeca:make_sdr_mango:301 - waiting 2 minutes for the sdr-09 to change design.
2023-09-01 08:05:06.410 | SUCCESS  | chi.expeca:make_sdr_mango:315 - SERVICE=change_design env variable is set
running change_design...
You have chosen sdr-09 and design mango
sdr-09 is reachable: {'mango': {'ip': '10.30.1.17', 'port': '22', 'tenant-port': 'te1/0/26', 'up': False}, 'ni': {'ip': '10.30.1.18', 'port': '22', 'tenant-port': 'te4/0/13', 'up': True}}
SSHing to 10.30.1.18, username:root, password: 
running command on 10.30.1.18: 
cp /uboot/mango_bootbin/BOOT.bin /uboot/ ; /sbin/reboot > /dev/null 2>&1 &
command stdout: 

command sdterr: 

Waiting 200 seconds for the new desgin to load
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
sdr-09 design has be

## Reboot the SDRs

In [ ]:
# reset sdr using the reserved worker
sdrnet = chi.network.get_network(sdr_name+"-rj45-net")
restart_sdr(sdr_name,sdrnet['id'],worker_reservation_id,"ens1")


2023-09-01 10:10:24.442 | SUCCESS  | chi.expeca:restart_sdr:255 - created reboot-sdr container.
2023-09-01 10:10:24.445 | INFO     | chi.expeca:restart_sdr:257 - waiting 2 minutes for the sdr-09 to reboot.
2023-09-01 10:11:15.129 | SUCCESS  | chi.expeca:restart_sdr:267 - SERVICE=reboot env variable is set
running reboot...
You have chosen sdr-09
sdr-09 is reachable: {'mango': {'ip': '10.30.1.17', 'port': '22', 'tenant-port': 'te1/0/26', 'up': True}, 'ni': {'ip': '10.30.1.18', 'port': '22', 'tenant-port': 'te4/0/13'}}
SSHing to 10.30.1.17, username:root, password: root
running command on 10.30.1.17: 
/sbin/reboot > /dev/null 2>&1 &
Waiting 200 seconds for the sdr to load
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
sdr-09 mango is up again.

2023-09-01 10:11:15.612 | INFO     | chi.expeca:wait_until_container_removed:30 - waiting 30 seconds for reboot-sdr container to be removed
2023-09-01 10:11:20.836 | INFO

# Run A Public Container to Test SDR

Run a public container to test the SFP port.

In [ ]:
# check public IPs and select one
available_pub_ips = get_available_publicips()
pub_ip = available_pub_ips[0]
logger.info(f"Available public ips: {available_pub_ips}.")
logger.info(f"We choose {pub_ip} for this container.")

# run the container
sdrsfpnet = chi.network.get_network(sdr_name+"-sfp-net")
publicnet = chi.network.get_network("serverpublic")
container_name = "my-public-sdr-host"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/sshd-image",
    reservation_id = worker_reservation_id,
    environment = {
        "DNS_IP":"8.8.8.8",
        "GATEWAY_IP":"130.237.11.97",
        "PASS":"expeca"
    },
    mounts = [],
    nets = [
        { "network" : publicnet['id'] },
        { "network" : sdrsfpnet['id'] },
    ],
    labels = {
        "networks.1.interface":"ens1",
        "networks.1.ip":pub_ip+"/27",
        "networks.1.gateway":"130.237.11.97",
        "capabilities.privileged":"true",
        "networks.2.interface":"eno12409",
        "networks.2.ip":"10.30.10.120/24",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container, reachable at {pub_ip}.")

2024-07-10 16:59:28.391 | SUCCESS  | __main__:<cell line: 28>:28 - created my-public-sdr-host container.
